In [25]:
import json
import datasets

In [2]:
# load dataset
all_data = datasets.load_dataset('wikisql')

Using custom data configuration default
Reusing dataset wikisql (/Users/alexwang/.cache/huggingface/datasets/wikisql/default/0.1.0/7037bfe6a42b1ca2b6ac3ccacba5253b1825d31379e9cc626fc79a620977252d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
#all_data['train'][0]['table']
all_data['train'][0]['table']

{'header': ['State/territory',
  'Text/background colour',
  'Format',
  'Current slogan',
  'Current series',
  'Notes'],
 'page_title': '',
 'page_id': '',
 'types': ['text', 'text', 'text', 'text', 'text', 'text'],
 'id': '1-1000181-1',
 'section_title': '',
 'caption': '',
 'rows': [['Australian Capital Territory',
   'blue/white',
   'Yaa·nna',
   'ACT · CELEBRATION OF A CENTURY 2013',
   'YIL·00A',
   'Slogan screenprinted on plate'],
  ['New South Wales',
   'black/yellow',
   'aa·nn·aa',
   'NEW SOUTH WALES',
   'BX·99·HI',
   'No slogan on current series'],
  ['New South Wales',
   'black/white',
   'aaa·nna',
   'NSW',
   'CPX·12A',
   'Optional white slimline series'],
  ['Northern Territory',
   'ochre/white',
   'Ca·nn·aa',
   'NT · OUTBACK AUSTRALIA',
   'CB·06·ZZ',
   'New series began in June 2011'],
  ['Queensland',
   'maroon/white',
   'nnn·aaa',
   'QUEENSLAND · SUNSHINE STATE',
   '999·TLG',
   'Slogan embossed on plate'],
  ['South Australia',
   'black/white',
  

## Format Data

In [13]:
# NB: model must be cased
# NB: position of question vs table

def format_input(example, input_token="<i>", col_tok="<c>", typ_tok="<t>"):
    # input_token: string for separating table from question
    question = example['question']
    
    table = example['table']
    table_name = table['name'] if table['name'] else table['id']
    cols = table['header']
    types = table['types']
    linearized_table = f'{table_name} '
    for col, typ in zip(cols, types):
        linearized_table += f'{col_tok} {col} {typ_tok} {typ} '
    
    return f"{linearized_table} {input_token} {question}"
    
    
def format_output(example):
    target = example['sql']['human_readable']
    
    table = example['table']
    table_name = table['name'] if table['name'] else table['id']
    target = target.replace('FROM table', f'FROM {table_name}')
    return target
    
    
def format_example(example, sep_token="<s>", input_token="<i>",
                   col_tok="<c>", typ_tok="<t>"):
    example_input = format_input(example, input_token=input_token)
    example_output = format_output(example)
    return f"{example_input} {sep_token} {example_output}"


def process_dataset(data, max_examples=-1,
                    example_separator="--SEPARATOR--"):
    """
    """
    n_exs = 0
    processed_examples = []
    for example in data:
        processed_examples.append(format_example(example))
        n_exs += 1
        if max_examples > 0 and n_exs >= max_examples:
            break
            
    print(f"\tProcessed {len(processed_examples)} examples")
    return f"\n{example_separator}\n".join(processed_examples)

In [16]:
SPLITS = ["train", "validation", "test"]
max_examples=10000

for split in SPLITS:
    split_data = all_data[split]
    print(f"Processing {split}")
    processed_data = process_dataset(split_data, max_examples=max_examples)
    with open(f'data/{split}.nexs{max_examples}.txt', 'w') as out_fh:
        out_fh.write(processed_data)

Processing train
	Processed 10000 examples
Processing validation
	Processed 8421 examples
Processing test
	Processed 10000 examples


## Extract Tables

In [33]:
def extract_tables(data):
    """ 
    """
    
    all_tables = {}
    
    for split in ["train", "validation", "test"]:
        print(f"Extracting tables from {split}")
        for example in data[split]:
            table = example["table"]
            table_name = table['name'] if table['name'] else table['id']
            
            if table_name in all_tables:
                assert all_tables[table_name] == table, "Table name collision"
            else:
                all_tables[table_name] = table
            
    all_tables = list(all_tables.values())
    with open("data/all_tables.jsonl", 'w', encoding='utf-8') as out_fh:
        for table in all_tables:
            out_fh.write(f'{json.dumps(table)}\n')
            
    print(f"Wrote {len(all_tables)} tables")
    
extract_tables(all_data)

Extracting tables from train
Extracting tables from validation
Extracting tables from test
Wrote 25683 tables


In [29]:
# try loading 
tables = [json.loads(l) for l in open("data/all_tables.jsonl")]

In [30]:
tables[0]

{'header': ['Player',
  'No.',
  'Nationality',
  'Position',
  'Years in Toronto',
  'School/Club Team'],
 'page_title': 'Toronto Raptors all-time roster',
 'page_id': '',
 'types': ['text', 'text', 'text', 'text', 'text', 'text'],
 'id': '1-10015132-11',
 'section_title': 'L',
 'caption': 'L',
 'rows': [['Antonio Lang',
   '21',
   'United States',
   'Guard-Forward',
   '1999-2000',
   'Duke'],
  ['Voshon Lenard', '2', 'United States', 'Guard', '2002-03', 'Minnesota'],
  ['Martin Lewis',
   '32, 44',
   'United States',
   'Guard-Forward',
   '1996-97',
   'Butler CC (KS)'],
  ['Brad Lohaus', '33', 'United States', 'Forward-Center', '1996', 'Iowa'],
  ['Art Long',
   '42',
   'United States',
   'Forward-Center',
   '2002-03',
   'Cincinnati'],
  ['John Long', '25', 'United States', 'Guard', '1996-97', 'Detroit'],
  ['Kyle Lowry', '3', 'United States', 'Guard', '2012-Present', 'Villanova']],
 'name': 'table_10015132_11'}